In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

from after_pt.reward_model import RewardModel

login(token='<>')

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")



/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.43s/it]


In [2]:
import sys, os
project_root = os.path.abspath('/Users/subhojit/workspace/saturn/src')
if project_root not in sys.path:
    sys.path.append(project_root)

from after_pt import *

In [3]:
device = "mps"
model = model.to(device)

In [4]:
text = """<s>[INST] Who is Kurt Godel ? [/INST]"""
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

In [5]:
model_inputs = encodeds.to(device)


In [6]:
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>[INST] Who is Kurt Godel ? [/INST] Kurt Godel (1906-1978) was a German logician and mathematician, known for his work in axiomatic set theory, proof theory, and mathematical logic. Godel is famous for his Godel's incompleteness theorems, which show that within any axiomatic system that contains basic arithmetic, there will always be true statements that cannot be proven within that system. Godel's work also introduced new definitions and methods in mathematical logic, and helped to lay the foundation for the development of modern computability theory.</s>


In [17]:
import torch.nn as nn
import torch

class RewardModelLite1(nn.Module):
    def __init__(self, base_model, tokenizer):
        super().__init__()
        device = "mps"
        self.base_model = base_model.to(device)
        self.tokenizer = tokenizer
        # scalar reward head
        self.reward_head = nn.Linear(self.base_model.config.hidden_size, 1, device=device)

    def forward(self, input_ids, attention_mask):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = outputs.hidden_states[-1]
        last_token_idx = attention_mask.sum(dim=1) - 1  # (B,)
        last_token_hidden = last_hidden_state[torch.arange(last_hidden_state.size(0)), last_token_idx]  # (B, D)
        reward = self.reward_head(last_token_hidden).squeeze(-1)  # (B,)
        return reward

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

text = "<s>[INST] What is dropout in neural networks? [/INST] Dropout is a regularization technique to switch off neuron randomly"
inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

reward_model = RewardModelLite1(model, tokenizer)
reward = reward_model(**inputs)  # scalar for each example
print(reward)


tensor([-2.8087], device='mps:0', grad_fn=<SqueezeBackward1>)
